Establish connection

In [1]:
%reload_ext google.cloud.bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\\Users\\Cermati\\AppData\\Roaming\\gcloud\\application_default_credentials.json"
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

C:\Users\Cermati\AppData\Roaming\gcloud\application_default_credentials.json


Insert query here

In [31]:
%%bigquery param1 --project athena-179008

with data as (
    select
        lcl.loan_contract_id as loan_id
        , ir.borrower_id as user_id
        , CASE trim(upper(coalesce(a.currentCompanyAddressProvince,a.applicantCurrentResidenceProvince,a.applicantResidenceProvince)))
            WHEN 'NANGGROE ACEH DARUSSALAM' THEN 'ACEH'
            WHEN 'KEPULA TAN RIAU' THEN 'KEPULAUAN RIAU'
            WHEN 'JAWA TIMUR\nKOTA SURABAYA\n3576396206810002\nWW.A\nA ASTA\n PROVINSI JAWA TIMUR' THEN 'JAWA TIMUR'
            WHEN 'IMUR' THEN 'JAWA TIMUR'
            WHEN 'JAWA IINUI' THEN 'JAWA TIMUR'
            WHEN 'JAWA TARAT' THEN 'JAWA BARAT'
            WHEN '1 AA' THEN 'JAWA BARAT'
            WHEN 'DAERAH ISTIMEWA YOGYAKARTA' THEN 'DI YOGYAKARTA'
            WHEN 'BANTEN\nKOTA TANGERANG\n*3172016 1143000\nNIK\n PROVINS BANTEN' THEN 'BANTEN'
            WHEN 'NUSA TENGGARAT' THEN 'NUSA TENGGARA TIMUR'
            WHEN 'KALIMANTAN BARAT\nKOTA PONTIANAK\nWIK\nL371035706AL00O3\n PROVINS KALIMANTAN BARAT' THEN 'KALIMANTAN BARAT'
            ELSE trim(upper(coalesce(a.currentCompanyAddressProvince,a.applicantCurrentResidenceProvince,a.applicantResidenceProvince)))
        END as province
        , COALESCE(DATE_DIFF(current_date,date(a.applicantDateOfBirth), YEAR) - IF(EXTRACT(MONTH FROM date(a.applicantDateOfBirth))*100 + EXTRACT(DAY FROM date(a.applicantDateOfBirth)) > EXTRACT(MONTH FROM current_date)*100 + EXTRACT(DAY FROM current_date),1,0), u.age)  as age
        , a.currentCompanyIncomePerMonth as income_per_month
        , CASE a.currentCompanyIndustry
            WHEN 'Asuransi/Forex' THEN 'Financial Services' 
            WHEN 'Biro Jasa' THEN 'Services' 
            WHEN 'General & Wholesale Trading' THEN 'Trading/Warehouse' 
            WHEN 'Hukum dan Perpajakan' THEN 'Law and Taxation' 
            WHEN 'Industri/Pabrik' THEN 'Industrial/Factory' 
            WHEN 'Jasa/Pelayanan' THEN 'Services' 
            WHEN 'Kesehatan/Klinik' THEN 'Health Services' 
            WHEN 'Keuangan' THEN 'Financial Services' 
            WHEN 'Keuangan/Bank' THEN 'Financial Services' 
            WHEN 'Konsultan' THEN 'Consultancy ' 
            WHEN 'Kontraktor/Properti' THEN 'Contractor/Real Estate' 
            WHEN 'Medis' THEN 'Health Services' 
            WHEN 'Pariwisata' THEN 'Leisure' 
            WHEN 'Pemerintahan' THEN 'Government Sector' 
            WHEN 'Perdagangan' THEN 'Trading/Warehouse' 
            WHEN 'Pergudangan' THEN 'Trading/Warehouse' 
            WHEN 'Perkebunan' THEN 'Agriculture' 
            WHEN 'Pertambangan' THEN 'Mining' 
            WHEN 'Properti' THEN 'Contractor/Real Estate' 
            WHEN 'Restoran/Bar/Cafe' THEN 'Hospitality' 
            WHEN 'Retail' THEN 'Retail' 
            WHEN 'Spesialis' THEN 'Professional' 
            WHEN 'Telekomunikasi' THEN 'Telecommunication' 
            WHEN 'Transportasi' THEN 'Transportation' 
            WHEN 'perdagangan' THEN 'Trading/Warehouse' 
            WHEN 'service' THEN 'Services' 
            ELSE 'Others' 
        END as company_industry
        , DATE(lc.approved_date,"Asia/Jakarta") as disbursed_date
        , lc.approved_date
        , ra.latest_model_score_norescore as score
        , lc.tenure
        , CASE 
            WHEN ir.entity_id like 'ATH-%' THEN ir.type
            ELSE 'PAYLATER'
        END as product
        , lc.loan_amount
        , COALESCE(date_diff(Current_Date(),date(min(case when lcl.status in ('UNPAID','PARTIALLY_PAID') then lcl.due_date end),"Asia/Jakarta"),day),0) as dpd
        , sum(case when lcl.type='PRINCIPAL' then ROUND(lcl.balance/lc.principal*lc.loan_amount,0) else 0 end) as balance_loan_amount
        
        , SUM(
            CASE
                WHEN lcl.type='PRINCIPAL' 
                AND COALESCE(date_diff(
                            Current_Date()
                            , date(case when lcl.status in ('UNPAID','PARTIALLY_PAID') then lcl.due_date end,"Asia/Jakarta")
                            , DAY
                        ),0) <= 120
                    THEN ROUND(lcl.balance/lc.principal*lc.loan_amount,0)
                    ELSE 0
            END
        ) as balance_loan_amount_after_writeoff_120
        
        , SUM(
            CASE
                WHEN lcl.type='PRINCIPAL' 
                AND COALESCE(date_diff(
                            Current_Date()
                            , date(case when lcl.status in ('UNPAID','PARTIALLY_PAID') then lcl.due_date end,"Asia/Jakarta")
                            , DAY
                        ),0) <=0
                    THEN ROUND(lcl.balance/lc.principal*lc.loan_amount,0)
                    ELSE 0
            END
        ) as balance_loan_amount_current
        
        , SUM(
            CASE
                WHEN lcl.type='PRINCIPAL' 
                AND COALESCE(date_diff(
                            Current_Date()
                            , date(case when lcl.status in ('UNPAID','PARTIALLY_PAID') then lcl.due_date end,"Asia/Jakarta")
                            , DAY
                        ),0) BETWEEN 1 AND 30
                    THEN ROUND(lcl.balance/lc.principal*lc.loan_amount,0)
                    ELSE 0
            END
        ) as balance_loan_amount_1_30
        
        
        , SUM(
            CASE
                WHEN lcl.type='PRINCIPAL' 
                AND COALESCE(date_diff(
                            Current_Date()
                            , date(case when lcl.status in ('UNPAID','PARTIALLY_PAID') then lcl.due_date end,"Asia/Jakarta")
                            , DAY
                        ),0) BETWEEN 31 AND 60
                    THEN ROUND(lcl.balance/lc.principal*lc.loan_amount,0)
                    ELSE 0
            END
        ) as balance_loan_amount_31_60
        
        
        , SUM(
            CASE
                WHEN lcl.type='PRINCIPAL' 
                AND COALESCE(date_diff(
                            Current_Date()
                            , date(case when lcl.status in ('UNPAID','PARTIALLY_PAID') then lcl.due_date end,"Asia/Jakarta")
                            , DAY
                        ),0) BETWEEN 61 AND 90
                    THEN ROUND(lcl.balance/lc.principal*lc.loan_amount,0)
                    ELSE 0
            END
        ) as balance_loan_amount_61_90
        
        , SUM(
            CASE
                WHEN lcl.type='PRINCIPAL' 
                AND COALESCE(date_diff(
                            Current_Date()
                            , date(case when lcl.status in ('UNPAID','PARTIALLY_PAID') then lcl.due_date end,"Asia/Jakarta")
                            , DAY
                        ),0) BETWEEN 91 AND 120
                    THEN ROUND(lcl.balance/lc.principal*lc.loan_amount,0)
                    ELSE 0
            END
        ) as balance_loan_amount_91_120
        
        , SUM(
            CASE
                WHEN lcl.type='PRINCIPAL' 
                AND COALESCE(date_diff(
                            Current_Date()
                            , date(case when lcl.status in ('UNPAID','PARTIALLY_PAID') then lcl.due_date end,"Asia/Jakarta")
                            , DAY
                        ),0) > 120
                    THEN ROUND(lcl.balance/lc.principal*lc.loan_amount,0)
                    ELSE 0
            END
        ) as balance_loan_amount_120plus
    from data-platform-indodana.vayu.indodana_credit_loan_contract_ledgers lcl
    INNER JOIN data-platform-indodana.vayu.indodana_credit_loan_contracts lc ON lc.id=lcl.loan_contract_id
    LEFT JOIN athena-179008.vayu_data_mart.view_trx_contracts c ON c.loan_contract_id=lc.id
    INNER JOIN data-platform-indodana.vayu.indodana_credit_installment_requests ir ON ir.id=lc.installment_request_id
    LEFT JOIN athena-179008.vayu_data_mart.indodanamarketing_cli_application cli ON cli.credit_limit_account_id=ir.credit_limit_account_id
    INNER JOIN data-platform-indodana.vayu.indodana_athena_applications a ON COALESCE(cli.orderId,ir.entity_id)=a.orderId
    LEFT JOIN athena-179008.vayu_data_mart.dm_application_risk_analytics ra ON ra.orderId=a.orderId
    INNER JOIN athena-179008.vayu_data_mart.indodanamarketing_user_segmentation u ON u.userId=ir.borrower_id
    where lcl.status not in ('CANCELLED') AND lc.status='ACTIVE' AND ir.type<>'LAPTOP_OWNERSHIP_PROGRAM'
    group by 1,2,3,4,5,6,7,8,9,10,11,12, lc.principal
)

SELECT 
    ROW_NUMBER() OVER(ORDER BY approved_date) as _no, * EXCEPT(approved_date)
FROM data
WHERE loan_id<>'6d595d84-e0e0-11e9-8cd1-093ae60d74e0'

Sample 5 rows to check

In [32]:
param1.shape
# param1.dtypes

(355836, 21)

In [33]:
param1.head()

,_no,loan_id,user_id,province,age,income_per_month,company_industry,disbursed_date,score,tenure,...,loan_amount,dpd,balance_loan_amount,balance_loan_amount_after_writeoff_120,balance_loan_amount_current,balance_loan_amount_1_30,balance_loan_amount_31_60,balance_loan_amount_61_90,balance_loan_amount_91_120,balance_loan_amount_120plus
0,1,064ae621-d70d-11ea-9b11-1b7e4f91faf8,faaa91c9-e332-4e64-954c-907d7d2ce07c,DKI JAKARTA,34,6100000,Services,2018-02-06,None,2,...,1500000,1155,778846,0,0,0,0,0,0,778846
1,2,d82cde61-d70c-11ea-9b11-1b7e4f91faf8,99b63468-8e3b-459d-bbd6-a63f1bb78276,JAWA BARAT,27,4200000,Consultancy,2018-02-07,None,2,...,1200000,1185,1200000,0,0,0,0,0,0,1200000
2,3,f38fd311-d70c-11ea-9b11-1b7e4f91faf8,77f67106-bf1a-42b9-88a1-a2ec376db9d8,DKI JAKARTA,29,6500000,Financial Services,2018-02-07,None,1,...,1000000,1185,637164,0,0,0,0,0,0,637164
3,4,ee655873-d70c-11ea-9b11-1b7e4f91faf8,ede25f12-2ea2-4f62-bcee-7c214891ba3b,DKI JAKARTA,32,4062000,Services,2018-02-07,None,1,...,500000,1185,500000,0,0,0,0,0,0,500000
4,5,f910f941-d70c-11ea-9b11-1b7e4f91faf8,d985fae5-1f2b-4576-8168-3bf16eac4f94,JAWA BARAT,40,8500000,Services,2018-02-07,None,1,...,500000,1179,500000,0,0,0,0,0,0,500000


In [39]:
print(param1.describe())
print(param1.isna().any())
print(param1.isnull().sum())
print(param1['product'][param1['dpd']==-355])

                 _no            age  income_per_month         tenure  \
count  355836.000000  355836.000000      3.558360e+05  355836.000000   
mean   177918.500000      32.493458      7.821676e+06       2.205609   
std    102721.149531       7.357387      1.195459e+07       1.978004   
min         1.000000      17.000000      0.000000e+00       1.000000   
25%     88959.750000      27.000000      4.000000e+06       1.000000   
50%    177918.500000      31.000000      5.850000e+06       1.000000   
75%    266877.250000      37.000000      8.000000e+06       3.000000   
max    355836.000000      68.000000      1.000000e+09      12.000000   

                 dpd  
count  355836.000000  
mean      144.492890  
std       269.121333  
min      -355.000000  
25%       -24.000000  
50%       -11.000000  
75%       320.000000  
max      1185.000000  
_no                                       False
loan_id                                   False
user_id                                   False


In [40]:
param1.groupby('province',dropna=False)['loan_id'].count()

province
ACEH                            221
BALI                           3841
BANTEN                        38421
BENGKULU                         88
DI YOGYAKARTA                  3374
DKI JAKARTA                  161001
GORONTALO                        42
JAMBI                           416
JAWA BARAT                    86814
JAWA TENGAH                    7851
JAWA TIMUR                    31695
KALIMANTAN BARAT                876
KALIMANTAN SELATAN              413
KALIMANTAN TENGAH               132
KALIMANTAN TIMUR               1070
KALIMANTAN UTARA                 34
KEPULAUAN BANGKA BELITUNG        52
KEPULAUAN RIAU                 3135
LAMPUNG                         691
MALUKU                           62
MALUKU UTARA                     42
NUSA TENGGARA BARAT             149
NUSA TENGGARA TIMUR             152
PAPUA                             9
PAPUA BARAT                       4
RIAU                            764
SULAWESI BARAT                   34
SULAWESI SELATAN   

In [41]:
param1.groupby('company_industry',dropna=False)['loan_id'].count()

company_industry
Agriculture                1434
Consultancy                6723
Contractor/Real Estate    18100
Financial Services        25506
Government Sector          9930
Health Services           11761
Hospitality               13820
Industrial/Factory        36516
Law and Taxation           1142
Leisure                    6137
Mining                     3977
Others                    11012
Professional               2274
Retail                    54256
Services                  71756
Telecommunication         12175
Trading/Warehouse         58254
Transportation            11063
Name: loan_id, dtype: int64

In [42]:
param1.groupby('age',dropna=False)['loan_id'].count()

age
17       12
18       71
19      349
20     2950
21     5601
22     8298
23    12772
24    15524
25    17459
26    19814
27    19830
28    20034
29    20494
30    19425
31    18965
32    16790
33    16330
34    14952
35    14323
36    13438
37    12256
38    12369
39    10953
40    10080
41     8520
42     7225
43     6370
44     5604
45     4352
46     3883
47     3231
48     3001
49     2421
50     2003
51     1606
52     1302
53      863
54      533
55      473
56      363
57      261
58      158
59      138
60      164
61       86
62       47
63       34
64       32
65       73
66        3
68        1
Name: loan_id, dtype: int64

In [ ]:
# profession = param1.groupby('profession',dropna=False)['entity_id'].count().to_frame()
# profession.reset_index(inplace=True)
# profession.dtypes
# profession.to_excel('Testttt.xlsx',index=False)